# Notebook 2: fields

## Fields

- `host`: string that must match a valid IPv4 address
- `port`: int between 1~65535 (inclusive)
- `database`: string with a min length of 5 characters (leading/trailing whitespaces are cleared)

In [1]:
!cat 2*.py

from pydantic import BaseSettings, Field


class MySettings(BaseSettings):
    # https://www.regexpal.com/96770
    host: str = Field(..., regex="^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$")
    """Must be a valid IPv4 address"""
    port: int = Field(ge=1, le=65535)
    """Must be a valid port between 1~65535 (inclusive)"""
    database: str = Field(min_length=5)
    """Must be a string with at least 5 characters"""
    
    class Config:
        env_prefix = "APP_"
        env_file = ".env"
        anystr_strip_whitespace = True
        """For all strings, strip leading & trailing whitespaces"""


settings = MySettings()
print("App host:", settings.host)
print("App port:", settings.port)
print("App database:", settings.database)


In [2]:
# With valid fields (read from .env file)
!echo "APP_HOST=1.1.1.1" > .env
!echo "APP_PORT=8080" >> .env
!echo "APP_DATABASE=thingies" >> .env

!python 2*.py

App host: 1.1.1.1
App port: 8080
App database: thingies


In [3]:
# Host field is an invalid IP (the other fields are read from the .env file)
!APP_HOST=NotAnIPAddress python 2*.py

Traceback (most recent call last):
  File "2-fields.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
host
  string does not match regex "^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$" (type=value_error.str.regex; pattern=^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$)


In [4]:
# Port field is out of range
!echo "With port=0:"
!APP_PORT=0 python 2*.py

!echo "\nWith port=100000:"
!APP_PORT=100000 python 2*.py

With port=0:
Traceback (most recent call last):
  File "2-fields.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
port
  ensure this value is greater than or equal to 1 (type=value_error.number.not_ge; limit_value=1)

With port=100000:
Traceback (most recent call last):
  File "2-fields.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
port
  ensure this value is less than or equal to 65535 (type=value_error.number.not_le; limit_value=65535)


In [5]:
# Database field string length must be at least 5 characters long
!APP_DATABASE=1234 python 2*.py

Traceback (most recent call last):
  File "2-fields.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
database
  ensure this value has at least 5 characters (type=value_error.any_str.min_length; limit_value=5)


In [6]:
# The strings on this class strip leading/trailing whitespaces...
!APP_DATABASE="     the_database     " python 2*.py

App host: 1.1.1.1
App port: 8080
App database: the_database


In [7]:
# ...and the min string length is verified after striping whitespaces
!APP_DATABASE="   1234    " python 2*.py

Traceback (most recent call last):
  File "2-fields.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
database
  ensure this value has at least 5 characters (type=value_error.any_str.min_length; limit_value=5)
